In [ ]:
# Plot AARE and Thd
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Settings
influxdb_url = "http://localhost:8086"
token = "random_token"
org = "ORG"
bucket = "anomalies"
measurement = "inc_result-B3B-Standard"
start_time = "1997-04-10T00:00:00Z"

# Instantiate the QueryAPI
client = InfluxDBClient(url=influxdb_url, token=token, org=org)
query_api = client.query_api()

# Construct the Flux query
query = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Query the data
result_df = query_api.query_data_frame(query=query)

if not result_df.empty:
    # Convert 'T' to type integer
    result_df['T'] = result_df['T'].astype(int)

    # Rename columns and simplify dataframe
    result_df.rename(columns={'_time': 'time', '_measurement': 'measurement'}, inplace=True)
    selected_columns = ['time', 'measurement', 'T', 'actual_value', 'predicted_value']
    if 'AARE' in result_df.columns and 'Thd' in result_df.columns:
        selected_columns += ['AARE', 'Thd']
        result_df = result_df[selected_columns]
        
        # Plotting
        plt.figure(figsize=(10, 5), dpi=500)
        plt.plot(result_df['time'], result_df['AARE'], label='AARE', color='black')
        plt.plot(result_df['time'], result_df['Thd'], label='Threshold', color='red', linestyle='--')
        plt.legend()
        plt.xlabel('Timestamp')
        plt.ylabel('AARE')
        #plt.title('AARE and Threshold Over Time')
        plt.xticks(rotation=45, fontsize=6)  # Rotates X-axis labels to make them readable
        plt.gca().margins(x=0) # Remove margis on x-axis
        plt.grid(True)
        plt.tight_layout()  # Adjusts plot to ensure everything fits without overlap

        plt.savefig('result-CC2.pdf',format='pdf')

        plt.show()
        
    else:
        print("Required columns 'AARE' or 'Thd' not found in data")
else:
    print("No data found")


In [ ]:
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Settings
influxdb_url = "http://localhost:8086"
token = "random_token"
org = "ORG"
bucket = "system_state"
measurement = "CC2"
bucket_ano = "anomalies"
measurement2 = "inc_detection-CC2-MinMax"
measurement_labels = "labels_CC2-seq"
start_time = "1997-04-10T00:00:00Z"

# Instantiate the QueryAPI
client = InfluxDBClient(url=influxdb_url, token=token, org=org)
query_api = client.query_api()

# Construct the Flux queries
query1 = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''
query2 = f'''
from(bucket: "{bucket_ano}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement2}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''
labels_query = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{measurement_labels}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Query the data
result_df1 = query_api.query_data_frame(query=query1)
marker_df = query_api.query_data_frame(query=query2)
labels_df = query_api.query_data_frame(query=labels_query)

# Plotting the main data
if not result_df1.empty:
    plt.figure(figsize=(9, 5), dpi=500)
    plt.plot(result_df1['_time'], result_df1['value'], label='Value', color='black')

    # Plotting the markers
    if not marker_df.empty:
        plt.scatter(marker_df['_time'], marker_df['actual_value'], color='red', label='Marker', zorder=5)

    # Plotting the labels as vertical dotted lines
    if not labels_df.empty:
        for timestamp in labels_df['_time']:
            plt.axvline(x=timestamp, color='dodgerblue', linestyle=':', label='Label' if 'Label' not in plt.gca().get_legend_handles_labels()[1] else "")

    plt.legend()
    plt.xlabel('Timestamp')
    plt.ylabel('Value')
    plt.xticks(rotation=45, fontsize=8)
    plt.gca().margins(x=0)
    plt.grid(True)
    plt.tight_layout()

    # Save the plot as a PDF file
    plt.savefig('detection-CC2.pdf', format='pdf')

    plt.show()
else:
    print("No data found for the main query")
